# Probabilistic Models – Spring 2021
## Exercise Session 3
Feb 10rd 16.15

<span style="color:red">**Carmen Díez**</span>

### Instructions
Make sure the notebook produces correct results when ran sequentially starting from the first cell. You can ensure this by clearing all outputs (`Edit > Clear All Outputs`), running all cells (`Run > Run All Cells`), and finally correcting any errors.

To get points:
1. Submit your answers to the automatically checked Moodle test. 
 - You have 5 tries on the test: the highest obtained score will be taken into account.
 - For numerical questions the tolerance is $1\cdot10^{-4}$.
2. Submit this notebook containing your derivations to Moodle.

The idea in this exercise is to understand how a naive Bayes classifier and a hidden Markov model work, by implementing them yourself. Therefore, you should not use any libraries providing the models off-the-shelf. Also you should not copy-paste any solutions you might find on the Internet. Doing so you will risk losing the points for the exercise. You can, of course, compare the output of your implementation to any reference implementation.

## Exercise 1
***

Consider the following document corpus, where the first column specifies the class of the document and the second column is the document itself.


In [1]:
!cat data/corpus.txt

Class	Text
spam	FREE online !!! results
ham	results repository online FREE
spam	FREE online results FREE !!! registration
spam	!!! registration FREE !!! repository
ham	conference online registration conference !!!
ham	conference results repository results FREE


(a) Construct a multinomial naive Bayes classifier (NBC; p. 24, lecture 5) using the corpus as the training data. Use Laplace smoothing to address the problem of zero probabilities. 

Report in Moodle:
- $P(C=\text{ham})$,
- $P(X=\text{repository} \mid C=\text{spam})$.



(b) Use the NBC to calculate the posterior probability for each document that it is spam. 

Report in Moodle:
- the obtained value for the last document in the corpus.

(c) Finally, compute and report in Moodle the posterior probabilities for the following documents (not used in the training of the NBC) to be spam:

In [2]:
!cat data/new_emails.txt

Class	Text
?	FREE online conference !!! registration
?	conference registration results conference online !!!


For each question asking for a posterior probability, remember to normalize so that your posterior probabilities sum up to 1.

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Total count of words
data = 'data/corpus.txt'
f = open(data, 'r')
lines = f.readlines()[1:]
ham = []
spam = []
for line in lines:
    lSplit = line.split()
    if lSplit[0] == 'ham':
        ham += lSplit[1:]
    elif lSplit[0] == 'spam':
        spam += lSplit[1:]
NHam = 3
NSpam = 3
N = 6
words = list(set(ham+spam))
lenWords = len(words)
words

['repository',
 'online',
 'registration',
 'FREE',
 'results',
 '!!!',
 'conference']

In [5]:
columns = ['word','spam', 'ham']
df = pd.DataFrame(columns=columns)
for w in words:
    nS = spam.count(w)
    nH = ham.count(w)
    df = df.append({'word': w, 'spam': nS, 'ham': nH}, ignore_index=True)
df

,word,spam,ham
0,repository,1,2
1,online,2,2
2,registration,2,1
3,FREE,4,2
4,results,2,3
5,!!!,4,1
6,conference,0,3


In [6]:
denomHam = df['ham'].sum() + lenWords
denomSpam = df['spam'].sum() + lenWords

In [7]:
dfProbs = df.copy()
for i in df.index:
    dfProbs.at[i, 'ham'] = (df.at[i, 'ham'] + 1)/(denomHam)
    dfProbs.at[i, 'spam'] = (df.at[i, 'spam'] + 1)/(denomSpam)
dfProbs = dfProbs.append({'word': 'P(C)', 'spam': (NSpam+1)/(N+2), 'ham': (NHam+1)/(N+2)}, ignore_index=True)
dfProbs

,word,spam,ham
0,repository,0.090909,0.142857
1,online,0.136364,0.142857
2,registration,0.136364,0.095238
3,FREE,0.227273,0.142857
4,results,0.136364,0.190476
5,!!!,0.227273,0.095238
6,conference,0.045455,0.190476
7,P(C),0.5,0.5


In [8]:
def PWordCondClass(w, cl):
    return dfProbs[dfProbs['word'] == w][cl].item()
def PClass(cl):
    return dfProbs[dfProbs['word'] == 'P(C)'][cl].item()

In [9]:
print('P(C=Ham)=', PClass('ham'))

P(C=Ham)= 0.5


In [10]:
pRepoCondSpam = PWordCondClass('repository', 'spam')
print('P(X=repository|C=Spam)=', pRepoCondSpam)

P(X=repository|C=Spam)= 0.09090909090909091


In [11]:
def PClassCondData(cl, data):
    prob = 1
    for w in data:
        prob = prob*PWordCondClass(w, cl)
    return prob*PClass(cl)

def posteriorSpam(data):
    pSpamCondData = PClassCondData('spam', data)
    pHamCondData = PClassCondData('ham', data)
    print('P(C=Spam,D)=', pSpamCondData)
    print('P(C=Ham,D)=', pHamCondData)

    norm = 1/(pSpamCondData+pHamCondData)
    print('P(C=Spam|D)=', pSpamCondData*norm)
    print('P(C=Ham|D)=', pHamCondData*norm)

In [12]:
words2 = lines[-1].split()[1:]
posteriorSpam(words2)

P(C=Spam,D)= 8.731706105519368e-06
P(C=Ham,D)= 7.051735498216129e-05
P(C=Spam|D)= 0.11018056221333225
P(C=Ham|D)= 0.8898194377866676


In [13]:
data2 = 'data/new_emails.txt'
f2 = open(data2, 'r')
lines2 = f2.readlines()[1:]

In [14]:
words3 = lines2[0].split()[1:]
posteriorSpam(words3)

P(C=Spam,D)= 2.1829265263798422e-05
P(C=Ham,D)= 1.7629338745540322e-05
P(C=Spam|D)= 0.5532194007327792
P(C=Ham|D)= 0.4467805992672207


In [15]:
words4 = lines2[1].split()[1:]
posteriorSpam(words4)

P(C=Spam,D)= 5.953435981035933e-07
P(C=Ham,D)= 4.477292379819764e-06
P(C=Spam|D)= 0.11736375342023181
P(C=Ham|D)= 0.8826362465797684


## Exercise 2
***

**NB**: The exercise does not assume knowledge on molecular biology. The description shows one simplified use case for a hidden Markov model.

The central dogma of molecular biology (roughly) states that DNA is transcribed into RNA. The RNA is then translated into protein, and proteins perform most of the functions within the cell. A protein is composed of a string of amino acids. While there are about twenty commonly occuring amino acids, and they can be grouped many different ways, we will consider them all to belong to either the Aliphatic or Hydroxyl group (to simplify calculations). This sequence is called the primary structure of the protein. A key factor in understanding the function of a particular protein is its secondary structure, that is, how the amino acids behave in three dimensional space. For example, the following figure shows a cartoon rendering of the secondary structure of a protein.

![](secondarystructure.jpg)

As the figure suggests, two of the most important features of the seconary structure are $\alpha$ helices and $\beta$ sheets. However, it is difficult to assess the secondary structure without expensive experimental techniques such as crystallography. On the other hand, finding the sequence of amino acids in a protein sequence is inexpensive. Consequently, a goal of computational biology is to infer the secondary structure of a protein given its primary structure (the sequence of amino acids).

Suppose we conduct a set of costly crystallography experiments and discover the following secondary structures and associated amino acid sequences. $\alpha$ means $\alpha~\text{helix}$, $\beta$ means $\beta~\text{sheet}$, $\mathtt{A}$ means aliphatic and $\mathtt{H}$ means hydroxyl. For example, in the first protein (i.e., first row) the second $\beta$ corresponds to the third $\mathtt{A}$, because they are both in the third position.

In [16]:
!cat data/sequences.txt

αββααβββ AAAHHHAA
αααβααβα HHAAHHHA
βαααββα AAAAHAH
αββαααβαα AHAAAHAAH


(a) Use these sequences to calculate the parameters for a hidden Markov model (HMM) in which the state variables correspond to the secondary structures and the observations correspond to the amino acids. We will use $S_i$ and $O_i$ to denote the secondary structure and the amino acid in position $i$, respectively. Recall that for an HMM, we need:

- The prior probabilities for the first state, $S_1$.  In this case, use the observed counts of the first state in each sequence.
- The *emission* probabilities, that is, the probability of observing a particular amino acid given the secondary structure.
- The *transition* probabilities, that is, the probability of the next state given the current state.

In all cases, use Laplace smoothing, as was done in Exercise 1.

Report in Moodle:

- the prior probability $P(S_1 = \alpha)$,
- the emission probabilities $P(O_i = \mathtt{A} \mid S_i = \alpha)$ and $P(O_i = \mathtt{H} \mid S_i = \beta)$,
- the transition probabilities $P(S_{i+1} = \beta \mid S_i = \alpha)$ and $P(S_{i+1} = \alpha \mid S_i = \beta)$.

(b) Now, using the HMM, use the foward-backward algorithm to calculate the posterior probabilities over all the secondary structure states, given the following observed amino acid sequence:

- $\mathtt{AHHAAHAAHA}$

Report in Moodle:
- $P(S_6 = \alpha \mid O_{1:10})$, i.e., the posterior probability for the 6th secondary state (corresponding to the third $\mathtt{H}$ in the sequence) to be $\alpha$.

(c) Finally, implement and use the Viterbi algorithm to calculate the most likely sequence of secondary structure states for the sequence of observations used in (b).

Report in Moodle:
- The obtained sequence. If there are multiple sequences with equally high likelihood, report any one of them. To avoid possible problems with character encoding use a and b to denote $\alpha$ and $\beta$: if you obtain the sequence $\alpha\alpha\alpha\alpha\alpha\beta\beta\beta\beta\beta$ report it as aaaaabbbbb.

Again, remember to normalize so that the posterior probabilities sum up to 1.

In [17]:
data = 'data/sequences.txt'
f = open(data, 'r')
lines = f.readlines()

In [18]:
alpha = lines[0][0]
beta = lines[0][1]
secondary = [alpha, beta]
amino = ['A', 'H']

In [19]:
def countSubstrings(string, substring):
    string_size = len(string)
    substring_size = len(substring)
    count = 0
    for i in range(0,string_size-substring_size+1):
        if string[i:i+substring_size] == substring:
            count+=1
    return count

In [20]:
trans = {}
for line in lines:
    sec = line.split()[0]
    print(sec)
    for i in range(len(secondary)):
        for j in range(len(secondary)):
            substr = secondary[i]+secondary[j]
            count = countSubstrings(sec,substr)
            print(substr, count)
            if substr in trans:
                trans[substr] += count
            else:
                trans[substr] = count
trans

αββααβββ
αα 1
αβ 2
βα 1
ββ 3
αααβααβα
αα 3
αβ 2
βα 2
ββ 0
βαααββα
αα 2
αβ 1
βα 2
ββ 1
αββαααβαα
αα 3
αβ 2
βα 2
ββ 1


{'αα': 9, 'αβ': 7, 'βα': 7, 'ββ': 5}

In [21]:
columnsTrans = ['Xt+1', 'Xt', 'P(Xt+1|Xt)']
dfTrans = pd.DataFrame(columns=columnsTrans)
for i in range(2):
    for j in range(2):
        s1 = secondary[i]
        s2 = secondary[j]
        s2Other = secondary[(j+1)%2]
        prob = (trans[s1+s2]+1)/(trans[s1+s2]+trans[s1+s2Other]+2)
        dfTrans = dfTrans.append({'Xt+1': s2, 'Xt': s1, 'P(Xt+1|Xt)': prob}, ignore_index=True)
dfTrans

,Xt+1,Xt,P(Xt+1|Xt)
0,α,α,0.555556
1,β,α,0.444444
2,α,β,0.571429
3,β,β,0.428571


In [22]:
emis = {}
for line in lines:
    both = line.split()
    print(both)
    for i in range(len(both[0])):
        pair = both[0][i] + both[1][i]
        if pair in emis:
            emis[pair] += 1
        else:
            emis[pair] = 1
    print(emis)

['αββααβββ', 'AAAHHHAA']
{'αA': 1, 'βA': 4, 'αH': 2, 'βH': 1}
['αααβααβα', 'HHAAHHHA']
{'αA': 3, 'βA': 5, 'αH': 6, 'βH': 2}
['βαααββα', 'AAAAHAH']
{'αA': 6, 'βA': 7, 'αH': 7, 'βH': 3}
['αββαααβαα', 'AHAAAHAAH']
{'αA': 10, 'βA': 9, 'αH': 9, 'βH': 4}


In [23]:
columnsEmission = ['Et', 'Xt', 'P(Et|Xt)']
dfEmis = pd.DataFrame(columns=columnsEmission)
for i in range(2):
    for j in range(2):
        e = amino[i]
        eOther = amino[(i+1)%2]
        s = secondary[j]
        prob = (emis[s+e]+1)/(emis[s+e]+emis[s+eOther]+2)
        dfEmis = dfEmis.append({'Et': e, 'Xt': s, 'P(Et|Xt)': prob}, ignore_index=True)
dfEmis

,Et,Xt,P(Et|Xt)
0,A,α,0.523810
1,A,β,0.666667
2,H,α,0.476190
3,H,β,0.333333


In [24]:
def probEmis(e, x):
    return dfEmis[(dfEmis['Et'] == e) & (dfEmis['Xt'] == x)]['P(Et|Xt)'].item()
def probTrans(xt1,xt):
    return dfTrans[(dfTrans['Xt+1'] == xt1) & (dfTrans['Xt'] == xt)]['P(Xt+1|Xt)'].item()
def probIni(x):
    prob = (3+1)/(4+2) #(Nalpha ini + 1)/(number of chains + 2 possibilities of aminos)
    if x==alpha:
        return prob
    elif x == beta: 
        return 1-prob 

In [25]:
print('P(S1=alpha)=', probIni(alpha))
print('P(Oi=A|Si=alpha)=', probEmis('A', alpha))
print('P(Oi=H|Si=beta)=', probEmis('H', beta))
print('P(Xt+1=beta|Xt=alpha)=', probTrans(beta, alpha))
print('P(Xt+1=alpha|Xt=beta)=', probTrans(alpha, beta))

P(S1=alpha)= 0.6666666666666666
P(Oi=A|Si=alpha)= 0.5238095238095238
P(Oi=H|Si=beta)= 0.3333333333333333
P(Xt+1=beta|Xt=alpha)= 0.4444444444444444
P(Xt+1=alpha|Xt=beta)= 0.5714285714285714


In [26]:
#b
def forward(k, e):
    probs = [0]*k
    pAlpha = probEmis(e[0], alpha) * probIni(alpha)
    pBeta = probEmis(e[0], beta) * probIni(beta)
    denom = pAlpha+pBeta
    p = {alpha: pAlpha/denom, beta: pBeta/denom}
    probs[0] = p
    for i in range(1,k):
        pAlpha = probTrans(alpha, alpha)*probs[i-1][alpha]
        pAlpha += probTrans(alpha, beta)*probs[i-1][beta]
        pAlpha = pAlpha*probEmis(e[i], alpha)
        
        pBeta = probTrans(beta, alpha)*probs[i-1][alpha]
        pBeta += probTrans(beta, beta)*probs[i-1][beta]
        pBeta = pBeta*probEmis(e[i], beta)
        denom = pAlpha+pBeta
        p = {alpha: pAlpha/denom, beta: pBeta/denom}
        probs[i] = p
    return probs

In [27]:
e = 'AHHAAHAAHA'
forward(6, e)

[{'α': 0.6111111111111112, 'β': 0.38888888888888895},
 {'α': 0.6467661691542289, 'β': 0.35323383084577115},
 {'α': 0.6462408715975955, 'β': 0.3537591284024045},
 {'α': 0.5011882241189397, 'β': 0.4988117758810604},
 {'α': 0.5035269305301889, 'β': 0.496473069469811},
 {'α': 0.6483499374434045, 'β': 0.35165006255659553}]

In [28]:
def backward(t, k, e):
    num = t-k
    probs = [0]*(num)

    pAlpha = probTrans(alpha, alpha)*probEmis(e[-1], alpha)
    pAlpha += probTrans(beta, alpha)*probEmis(e[-1], beta)

    pBeta = probTrans(alpha, beta)*probEmis(e[-1], alpha)
    pBeta += probTrans(beta, beta)*probEmis(e[-1], beta)

    p = {alpha: pAlpha, beta: pBeta}
    probs[0] = p
    for i in range(1,num):
        pAlpha = probTrans(alpha, alpha)*probEmis(e[-i-1], alpha)*probs[i-1][alpha]
        pAlpha += probTrans(beta, alpha)*probEmis(e[-i-1], beta)*probs[i-1][beta]
    
        pBeta = probTrans(alpha, beta)*probEmis(e[-i-1], alpha)*probs[i-1][alpha]
        pBeta += probTrans(beta, beta)*probEmis(e[-i-1], beta)*probs[i-1][beta]
        
        p = {alpha: pAlpha, beta: pBeta}
        probs[i] = p
    return probs

In [29]:
backward(10, 6, e)

[{'α': 0.5873015873015873, 'β': 0.5850340136054422},
 {'α': 0.24204249601074995, 'β': 0.24338624338624337},
 {'α': 0.14255008947206813, 'β': 0.14198702072581781},
 {'α': 0.08355305863285473, 'β': 0.08323577419901595}]

In [30]:
pAlpha = backward(10,6,e)[3][alpha]*forward(6,e)[5][alpha]
pBeta = backward(10,6,e)[3][beta]*forward(6,e)[5][beta]

denom = pAlpha+pBeta
p = {alpha: pAlpha/denom, beta: pBeta/denom}
p

{'α': 0.6492168731901188, 'β': 0.3507831268098813}

In [31]:
#c
def viterbi(e):
    probs = [0]*len(e)
    
    pAlpha = probEmis(e[0], alpha)*probIni(alpha)
    pBeta = probEmis(e[0], beta)*probIni(beta)
    
    pathIni1 = ['a']
    pathIni2 = ['b']
    
    probs[0] = {alpha: pAlpha, beta: pBeta}
    for i in range(1, len(e)):
        arg1Alpha = probTrans(alpha, alpha)*probs[i-1][alpha]
        arg2Alpha = probTrans(alpha, beta)*probs[i-1][beta]
        pAlpha = probEmis(e[i], alpha)*np.max([arg1Alpha, arg2Alpha])
        
        arg1Beta = probTrans(beta, alpha)*probs[i-1][alpha]
        arg2Beta = probTrans(beta, beta)*probs[i-1][beta]
        pBeta = probEmis(e[i], beta)*np.max([arg1Beta, arg2Beta])
        
        mAlpha = np.argmax([arg1Alpha, arg2Alpha])
        mBeta = np.argmax([arg1Beta, arg2Beta])

        if mAlpha == 0:
            path1 = pathIni1 + ['a']
        elif mAlpha == 1:
            path1 = pathIni2 + ['a']
        if mBeta == 0:
            path2 = pathIni1 + ['b']
        elif mBeta == 1:
            path2 = pathIni2 + ['b']
        
        probs[i] = {alpha: pAlpha, beta: pBeta}
        pathIni1 = path1
        pathIni2 = path2
    
    m = np.argmax([probs[len(e)-1][alpha], probs[len(e)-1][beta]])
    if m==0:
        path = pathIni1
    if m==1:
        path = pathIni2
    return path 

''.join(viterbi(e))

'aaabaaabab'